In [1]:
#importing all the basic libraries
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib as mpt
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
#load data into pandas datframe to see the structure of the data and analyze it
#skipping completely balnk lines and ignoring the lines with not having proper format
AMEX_DATA = pd.read_csv('C:/Users/bnaveen3/Desktop/AV/train.csv',skip_blank_lines=True,
                      error_bad_lines=False)
#Loading historical logics to check if this is useful
AMEX_HIST_LOG=pd.read_csv('C:/Users/bnaveen3/Desktop/AV/historical_user_logs.csv',skip_blank_lines=True,
                      error_bad_lines=False)

In [3]:
AMEX_TRAIN_DATA=AMEX_DATA
#total count is 463291
print('total unique session ids-',len(AMEX_TRAIN_DATA['session_id'].unique()))
#here total session ids and unique session ids are same..so we can say session id as an primary key.
print('number of record with no clicks',AMEX_TRAIN_DATA[AMEX_TRAIN_DATA['is_click']==0]['session_id'].count())
print('number of sessions with clicks',AMEX_TRAIN_DATA[AMEX_TRAIN_DATA['is_click']==1]['session_id'].count())
print('percenatge of sessions clicked on ads is ',(31331/431960)*100,'%')

total unique session ids- 463291
number of record with no clicks 431960
number of sessions with clicks 31331
percenatge of sessions clicked on ads is  7.253217890545422 %


In [4]:
def check_for_null_values(AMEX_TRAIN_DATA):
    #check if there is any data missing
    for i in AMEX_TRAIN_DATA.columns:
        print('Null Data in column ',i,
              AMEX_TRAIN_DATA[AMEX_TRAIN_DATA[i].isna()]['session_id'].count())
    


In [5]:
#By loking at the above data column product_category_2 is having lot of Blank values user_group_id,gender,age_level user_depth have less blank values
#city_developement_index has lot of blanks
#check if transformation is required for the null values
#drawing box plots for each feature--will not be useful mostly because data is not spread across on almost all features
#AMEX_TRAIN_DATA['user_depth'].plot.box()
AMEX_TRAIN_DATA[1:10]
l=1
product_code=[]
product=[]
#create a mappig dataframe for products
for i in pd.unique(AMEX_TRAIN_DATA['product']):
    product_code.append(l)
    product.append(i)
    l=l+1
product_mapping=pd.DataFrame(dict(product = product, product_code = product_code))
campaign_code=[]
campaign_id=[]
j=1
#create a mappig dataframe for CAMPAIGN IDS
for i in pd.unique(AMEX_TRAIN_DATA['campaign_id']):
    campaign_code.append(j)
    campaign_id.append(i)
    j=j+1
camp_mapping=pd.DataFrame(dict(campaign_id = campaign_id, campaign_code = campaign_code))
#pd.unique(AMEX_TRAIN_DATA['product'])
webpage_code=[]
webpage_id=[]
k=1
#create a mappig dataframe for CAMPAIGN IDS
for i in pd.unique(AMEX_TRAIN_DATA['webpage_id']):
    webpage_code.append(k)
    webpage_id.append(i)
    k=k+1
web_mapping=pd.DataFrame(dict(webpage_id = webpage_id, webpage_code = webpage_code))


In [306]:
product_mapping
#making product as index so that it will be used to map later
product_mapping_dict=product_mapping.set_index('product')
#making a dictionary which will be used to map accordingly
product_mapping_dict=product_mapping_dict.to_dict()['product_code']
product_mapping_dict

{'A': 10,
 'B': 5,
 'C': 1,
 'D': 6,
 'E': 8,
 'F': 3,
 'G': 7,
 'H': 4,
 'I': 2,
 'J': 9}

In [307]:
#making product as index so that it will be used to map later
camp_mapping_dict=camp_mapping.set_index('campaign_id')
#making a dictionary which will be used to map accordingly
camp_mapping_dict=camp_mapping_dict.to_dict()['campaign_code']
camp_mapping_dict

{82320: 8,
 98970: 6,
 105960: 2,
 118601: 10,
 359520: 1,
 360936: 4,
 396664: 9,
 404347: 5,
 405490: 3,
 414149: 7}

In [308]:
#making product as index so that it will be used to map later
web_mapping_dict=web_mapping.set_index('webpage_id')
#making a dictionary which will be used to map accordingly
web_mapping_dict=web_mapping_dict.to_dict()['webpage_code']
web_mapping_dict

{1734: 7,
 6970: 5,
 11085: 2,
 13787: 1,
 28529: 9,
 45962: 6,
 51181: 8,
 53587: 4,
 60305: 3}

In [309]:
#replacing 
AMEX_HIST_LOG['product']=AMEX_HIST_LOG['product'].map(product_mapping_dict)
AMEX_HIST_LOG['action']=AMEX_HIST_LOG['action'].map({'view':1,'interest':2})


In [310]:
AMEX_HIST_VIEW_DATA=AMEX_HIST_LOG[AMEX_HIST_LOG['action']==1].groupby(['user_id','product']).count()
AMEX_HIST_INTEREST_DATA=AMEX_HIST_LOG[AMEX_HIST_LOG['action']==2].groupby(['user_id','product']).count()

In [311]:
#Replacing categories with respective integers
AMEX_TRAIN_DATA['product']=AMEX_TRAIN_DATA['product'].map(product_mapping_dict)
AMEX_TRAIN_DATA['webpage_id']=AMEX_TRAIN_DATA['webpage_id'].map(web_mapping_dict)
AMEX_TRAIN_DATA['campaign_id']=AMEX_TRAIN_DATA['campaign_id'].map(camp_mapping_dict)


In [312]:
#mapping gender 
AMEX_TRAIN_DATA['gender']=AMEX_TRAIN_DATA['gender']=AMEX_TRAIN_DATA['gender'].map({'Female':1,'Male':2,'':''})

In [313]:
AMEX_TRAIN_DATA=AMEX_TRAIN_DATA.join(AMEX_HIST_VIEW_DATA['action'],on=['user_id','product'],rsuffix='_view_count')
AMEX_TRAIN_DATA=AMEX_TRAIN_DATA.join(AMEX_HIST_INTEREST_DATA['action'],on=['user_id','product'],rsuffix='_interest_count')
#AMEX_HIST_INTEREST_DATA['action']

In [314]:
AMEX_TRAIN_DATA.columns=['session_id', 'DateTime', 'user_id', 'product', 'campaign_id',
       'webpage_id', 'product_category_1', 'product_category_2',
       'user_group_id', 'gender', 'age_level', 'user_depth',
       'city_development_index', 'var_1', 'is_click', 'action_view_count',
       'action_interest_count']

In [315]:
AMEX_TRAIN_DATA['action_view_count']=AMEX_TRAIN_DATA['action_view_count'].fillna(0)
AMEX_TRAIN_DATA['action_interest_count']=AMEX_TRAIN_DATA['action_interest_count'].fillna(0)

In [316]:
AMEX_HIST_LOG['action'].value_counts()

1    23854100
2      433434
Name: action, dtype: int64

In [317]:
check_for_null_values(AMEX_TRAIN_DATA)

Null Data in column  session_id 0
Null Data in column  DateTime 0
Null Data in column  user_id 0
Null Data in column  product 0
Null Data in column  campaign_id 0
Null Data in column  webpage_id 0
Null Data in column  product_category_1 0
Null Data in column  product_category_2 365854
Null Data in column  user_group_id 18243
Null Data in column  gender 18243
Null Data in column  age_level 18243
Null Data in column  user_depth 18243
Null Data in column  city_development_index 125129
Null Data in column  var_1 0
Null Data in column  is_click 0
Null Data in column  action_view_count 0
Null Data in column  action_interest_count 0


In [6]:
#looking at the above data...for now dropping product_category_2 and city_developement_index
#as we dont know if there is no actual value or data fetching issue
AMEX_TRAIN_DATA=AMEX_TRAIN_DATA.drop(['product_category_2','city_development_index'],axis=1)

In [319]:
#Handle the Null values and fit the data
#check_for_null_values(AMEX_TRAIN_DATA)
AMEX_TRAIN_DATA=AMEX_TRAIN_DATA.fillna(0)#.drop(AMEX_TRAIN_DATA[AMEX_TRAIN_DATA['user_group_id'].isnull()].index,axis=0)

In [320]:
check_for_null_values(AMEX_TRAIN_DATA)
#deleted all rows with nul values for now and checj the 
AMEX_TRAIN_DATA.count()

Null Data in column  session_id 0
Null Data in column  DateTime 0
Null Data in column  user_id 0
Null Data in column  product 0
Null Data in column  campaign_id 0
Null Data in column  webpage_id 0
Null Data in column  product_category_1 0
Null Data in column  user_group_id 0
Null Data in column  gender 0
Null Data in column  age_level 0
Null Data in column  user_depth 0
Null Data in column  var_1 0
Null Data in column  is_click 0
Null Data in column  action_view_count 0
Null Data in column  action_interest_count 0


session_id               463291
DateTime                 463291
user_id                  463291
product                  463291
campaign_id              463291
webpage_id               463291
product_category_1       463291
user_group_id            463291
gender                   463291
age_level                463291
user_depth               463291
var_1                    463291
is_click                 463291
action_view_count        463291
action_interest_count    463291
dtype: int64

In [321]:
#oversampling of 1
AMEX_TRAIN_DATA=AMEX_TRAIN_DATA.append(AMEX_TRAIN_DATA[AMEX_TRAIN_DATA['is_click']==1])
AMEX_TRAIN_DATA=AMEX_TRAIN_DATA.append(AMEX_TRAIN_DATA[AMEX_TRAIN_DATA['is_click']==1])
AMEX_TRAIN_DATA=AMEX_TRAIN_DATA.append(AMEX_TRAIN_DATA[AMEX_TRAIN_DATA['is_click']==1])

In [322]:
#having a 20% of test data
from sklearn.model_selection import train_test_split
AMEX_TRAIN_DATA, AMEX_TEST_DATA = train_test_split(AMEX_TRAIN_DATA, test_size=0.2)

In [323]:
#splitting target result from training data
AMEX_TRAIN_TARGET=AMEX_TRAIN_DATA['is_click']
AMEX_TRAIN_FEAT =AMEX_TRAIN_DATA.drop(['is_click'],axis=1)


In [324]:
#splitting target result from training data
AMEX_TEST_TARGET=AMEX_TEST_DATA['is_click']
AMEX_TEST_FEAT =AMEX_TEST_DATA.drop(['is_click'],axis=1)

In [325]:
AMEX_TRAIN_FEAT['YEAR']=AMEX_TRAIN_FEAT['DateTime'].str[0:4]
AMEX_TRAIN_FEAT['MONTH']=AMEX_TRAIN_FEAT['DateTime'].str[5:7]
AMEX_TRAIN_FEAT['DAY']=AMEX_TRAIN_FEAT['DateTime'].str[8:10]
AMEX_TRAIN_FEAT['HOUR']=AMEX_TRAIN_FEAT['DateTime'].str[11:13]
AMEX_TEST_FEAT['YEAR']=AMEX_TEST_FEAT['DateTime'].str[0:4]
AMEX_TEST_FEAT['MONTH']=AMEX_TEST_FEAT['DateTime'].str[5:7]
AMEX_TEST_FEAT['DAY']=AMEX_TEST_FEAT['DateTime'].str[8:10]
AMEX_TEST_FEAT['HOUR']=AMEX_TEST_FEAT['DateTime'].str[11:13]

In [326]:
AMEX_TRAIN_FEAT=AMEX_TRAIN_FEAT.drop(['DateTime'],axis=1)
AMEX_TEST_FEAT=AMEX_TEST_FEAT.drop(['DateTime'],axis=1)
#dropping the below 2 as it didnt give ay improvement at all. YEAR is not required as we are only considering 1 year data
AMEX_TRAIN_FEAT=AMEX_TRAIN_FEAT.drop(['action_view_count','action_interest_count','YEAR','session_id'],axis=1)
AMEX_TEST_FEAT=AMEX_TEST_FEAT.drop(['action_view_count','action_interest_count','YEAR','session_id'],axis=1)


In [338]:
#Handle the Null values and fit the data
#check_for_null_values(AMEX_TRAIN_DATA)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(AMEX_TRAIN_FEAT)
AMEX_TRAIN_FEAT_NORM=scaler.transform(AMEX_TRAIN_FEAT)
AMEX_TEST_FEAT_NORM=scaler.transform(AMEX_TEST_FEAT)

In [215]:
AMEX_TRAIN_FEAT

,user_id,product,campaign_id,webpage_id,product_category_1,user_group_id,gender,age_level,user_depth,var_1,MONTH,DAY,HOUR
98862,612640,4,3,3,3,3.0,2,3.0,3.0,1,07,03,08
389234,867836,2,10,9,4,3.0,2,3.0,3.0,1,07,06,21
416998,1062646,6,7,6,2,4.0,2,4.0,3.0,0,07,07,09
230900,663783,1,3,3,3,2.0,2,2.0,3.0,0,07,04,21
232432,706394,1,1,1,4,2.0,2,2.0,3.0,1,07,04,22
246163,59308,2,4,1,2,2.0,2,2.0,3.0,0,07,05,08
266406,78379,6,6,5,4,2.0,2,2.0,3.0,0,07,05,13
138518,276434,2,9,8,1,5.0,2,5.0,2.0,0,07,03,17
413179,678048,1,3,3,3,3.0,2,3.0,3.0,1,07,07,08
357657,988176,9,5,4,1,2.0,2,2.0,3.0,1,07,06,14


In [339]:
#Fit the model
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier(n_estimators=20)
model.fit(AMEX_TRAIN_FEAT_NORM,AMEX_TRAIN_TARGET)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [329]:
#accuracy for the transactions where click was done
print('clicked accuracy',model.score(AMEX_TEST_FEAT[AMEX_TEST_TARGET==1],AMEX_TEST_TARGET[AMEX_TEST_TARGET==1]))
#accuracy for the transactions where click was not done
print('not clicked accuracy',model.score(AMEX_TEST_FEAT[AMEX_TEST_TARGET==0],AMEX_TEST_TARGET[AMEX_TEST_TARGET==0]))

clicked accuracy 0.9994604640009592
not clicked accuracy 0.9414539946114086


In [340]:
#total accuracy 
#AMEX_TEST_RES=AMEX_TEST_FEAT
#AMEX_TEST_RES['PREDICTED']=model.predict(AMEX_TEST_FEAT_NORM)
model.score(AMEX_TEST_FEAT_NORM,AMEX_TEST_TARGET)

0.9624895621218558

In [293]:
AMEX_TEST_RES['ACTUAL']=AMEX_TEST_TARGET

ValueError: cannot reindex from a duplicate axis

In [182]:
AMEX_TEST_FEAT.columns

Index(['session_id', 'user_id', 'product', 'campaign_id', 'webpage_id',
       'product_category_1', 'user_group_id', 'gender', 'age_level',
       'user_depth', 'var_1', 'MONTH', 'DAY', 'HOUR', 'PREDICTED', 'ACTUAL'],
      dtype='object')

In [341]:
#get the results for test data set provided in the competetion
TEST_DATA = pd.read_csv('C:/Users/bnaveen3/Desktop/AV/test.csv',skip_blank_lines=True,
                      error_bad_lines=False)

In [342]:
NEW_TEST_DATA=TEST_DATA

In [343]:

#Replacing categories with respective integers
NEW_TEST_DATA['product']=NEW_TEST_DATA['product'].map(product_mapping_dict)
NEW_TEST_DATA['webpage_id']=NEW_TEST_DATA['webpage_id'].map(web_mapping_dict)
NEW_TEST_DATA['campaign_id']=NEW_TEST_DATA['campaign_id'].map(camp_mapping_dict)
#mapping gender 
NEW_TEST_DATA['gender']=NEW_TEST_DATA['gender'].map({'Female':1,'Male':2,'':''})
#AMEX_HIST_INTEREST_DATA['action']
NEW_TEST_DATA=NEW_TEST_DATA.drop(['product_category_2','city_development_index'],axis=1)
NEW_TEST_DATA=NEW_TEST_DATA.fillna(0)
NEW_TEST_DATA =NEW_TEST_DATA.drop(['session_id'],axis=1)
#NEW_TEST_DATA=NEW_TEST_DATA.drop(['user_id'],axis=1)
NEW_TEST_DATA['YEAR']=NEW_TEST_DATA['DateTime'].str[0:4]
NEW_TEST_DATA['MONTH']=NEW_TEST_DATA['DateTime'].str[5:7]
NEW_TEST_DATA['DAY']=NEW_TEST_DATA['DateTime'].str[8:10]
NEW_TEST_DATA['HOUR']=NEW_TEST_DATA['DateTime'].str[11:13]
NEW_TEST_DATA=NEW_TEST_DATA.drop(['DateTime'],axis=1)
NEW_TEST_DATA=NEW_TEST_DATA.drop(['YEAR'],axis=1)

In [345]:
#predict 
NEW_TEST_DATA_NORM=scaler.transform(NEW_TEST_DATA)
NEW_RESULTS=model.predict(NEW_TEST_DATA_NORM)
#NEW_TEST_DATA_NORM

In [346]:
TEST_DATA['is_click']=NEW_RESULTS
SOLUTION_FILE=TEST_DATA[['session_id','is_click']]

In [347]:
SOLUTION_FILE.to_csv(path_or_buf='C:/Users/bnaveen3/Desktop/AV/solution_file.csv',index= False)

In [302]:
TEST_DATA[TEST_DATA['is_click']==1].count()

session_id                7944
DateTime                  7944
user_id                   7944
product                   7944
campaign_id               7944
webpage_id                7944
product_category_1        7944
product_category_2        2804
user_group_id             7402
gender                    7402
age_level                 7402
user_depth                7402
city_development_index    5476
var_1                     7944
is_click                  7944
dtype: int64